# Packages

In [6]:
from utils import *

# Construction of the Admittance Matrix

In [7]:
shunt_admittance = {
   (2, 4): 0 + 1.72j,
    (2, 5): 0 + 0.88j,
    (4, 5): 0 + 0.44j,
}

impedance = {
    (1, 5): 0.00150 + 0.02j,
    (3, 4): 0.00075 + 0.01j,
    (2, 4): 0.0090 + 0.1j,
    (2, 5): 0.0045 + 0.050j,
    (4, 5): 0.00225 + 0.025j,
}
nodes = sorted(set(i for par in impedance for i in par))
node_map = {no: idx for idx, no in enumerate(nodes)}
Ybus = compute_y_bus(impedance,shunt_admittance)
df_Ybus = pd.DataFrame(Ybus, index=[f"Bus {i}" for i in nodes], columns=[f"Bus {i}" for i in nodes])
df_Ybus = df_Ybus.round(4)
df_Ybus

,Bus 1,Bus 2,Bus 3,Bus 4,Bus 5
Bus 1,3.7290-49.7203j,0.0000+ 0.0000j,0.0000+ 0.0000j,0.0000+ 0.0000j,-3.7290+ 49.7203j
Bus 2,0.0000+ 0.0000j,2.6783-28.4590j,0.0000+ 0.0000j,-0.8928+ 9.9197j,-1.7855+ 19.8393j
Bus 3,0.0000+ 0.0000j,0.0000+ 0.0000j,7.4580-99.4406j,-7.4580+ 99.4406j,0.0000+ 0.0000j
Bus 4,0.0000+ 0.0000j,-0.8928+ 9.9197j,-7.4580+99.4406j,11.9219-147.9589j,-3.5711+ 39.6786j
Bus 5,-3.7290+49.7203j,-1.7855+19.8393j,0.0000+ 0.0000j,-3.5711+ 39.6786j,9.0856-108.5782j


# Power Flow (Newton-Raphson Method)

In [8]:
bus_types = [0, 2, 1, 2, 2]   # 0=Slack, 1=PV, 2=PQ
V = np.array([1.0, 1.0, 1.05,1.0,1.0])
theta = np.radians([0.0, 0.0, 0.0, 0.0, 0.0])
P_spec = np.array([0,-8,4.4,0,0])
Q_spec = np.array([0,-2.8,-0.4,0,0])
V_final, theta_final, P_calc, Q_calc = newton_power_flow(
    Ybus, bus_types, P_spec, Q_spec, V,
    baseMVA=100, tol_MVA=0.1, max_iter=20
)
voltages_df = pd.DataFrame({
    'Magnitude (pu)': [round(v, 4) for v in V_final],
    'Angle (deg)': [round(np.degrees(t), 4) for t in theta_final]
}, index=[f'Bus {i+1}' for i in range(len(V))])

print("Final voltages:")
display(voltages_df)

Iter 1: max mismatch = 800.000000 MVA
Iter 2: max mismatch = 107.908146 MVA
Iter 3: max mismatch = 15.578760 MVA
Iter 4: max mismatch = 0.851865 MVA
Iter 5: max mismatch = 0.002248 MVA
Converged within MVA tolerance.
Final voltages:


,Magnitude (pu),Angle (deg)
Bus 1,1.0000,0.0000
Bus 2,0.8338,-22.4063
Bus 3,1.0500,-0.5973
Bus 4,1.0193,-2.8340
Bus 5,0.9743,-4.5479


In [9]:
power_df = pd.DataFrame({
    'P (pu)': [round(p, 4) for p in P_calc],
    'Q (pu)': [round(q, 4) for q in Q_calc]
}, index=[f'Bus {i+1}' for i in range(len(P_calc))])

print("Power injections at all buses:")
display(power_df)

Power injections at all buses:


,P (pu),Q (pu)
Bus 1,3.9484,1.1428
Bus 2,-8.0000,-2.8000
Bus 3,4.4000,2.9748
Bus 4,-0.0000,0.0000
Bus 5,-0.0000,0.0000


In [10]:
losses = compute_loss(V_final, theta_final, impedance, shunt_admittance, node_map)

losses_df = pd.DataFrame({
    'P_loss (pu)': [round(p, 6) for _, _, p, _ in losses],
    'Q_loss (pu)': [round(q, 6) for _, _, _, q in losses]
}, index=[f'Line {i}–{k}' for i, k, _, _ in losses])

print("Losses per line:")
display(losses_df)

Losses per line:


,P_loss (pu),Q_loss (pu)
Line 1–5,0.025344,0.337914
Line 3–4,0.019190,0.255866
Line 2–4,0.118411,-0.175688
Line 2–5,0.175030,1.221238
Line 4–5,0.010409,-0.321753
